# Oscillation and Poisson Input

## setup

In [ ]:
import os

In [ ]:
base_path = os.path.join("..","data")

## utilities 

In [ ]:
import sys
sys.path.insert(0, "..")
import numpy as np

from persistence import FileMap

In [ ]:

from brian2 import ms, mV, kHz

from BrianExperiment import BrianExperiment
from network import NeuronPopulation, PoissonDeviceGroup, Connector
from distribution import draw_uniform
from differential_equations.neuron_equations import (
    eqs_P,
    eqs_I,
    PreEq_AMPA,
    PreEq_AMPA_pois,
    PreEq_GABA,
)
from differential_equations.neuron_parameters import delay_AMPA, delay_GABA, eL_p, eL_i

def run_exp(path:str, sim_time:float, rate_poisson_e:float, rate_poisson_i:float, report: bool=True, progress_bar: bool=True):
    """
    run a brian experiment with an E I network

    :param rate_poisson_e: rate of the homogeneous poisson input to the E population
    :param rate_poisson_i: rate of the homogeneous poisson input to the I population
    """

    with BrianExperiment(
        report_progress=report,
        progress_bar=progress_bar,
        persist=True,
        path=path,
        # object_path="/run_x/data",
        # neuron_eqs=["eqs_P", "PreEq_AMPA"],
        # neuron_params=["delay_AMPA"],
    ) as exp:

        # EI network

        # populations
        E = NeuronPopulation(
            4000, eqs_P, threshold="v_s>-30*mV", refractory=1.3 * ms, method="rk4"
        )

        I = NeuronPopulation(
            1000, eqs_I, threshold="v>-30*mV", refractory=1.3 * ms, method="rk4"
        )

        # synapses
        connect = Connector(synapse_type="static")

        S_E_E = connect(
            E,
            E,
            E.ids,
            E.ids,
            connect=("bernoulli", {"p": 0.01}),
            on_pre=PreEq_AMPA,
            delay=delay_AMPA,
        )
        # S_E_E.monitor(S_E_E.synapses, ["x_AMPA"])
        S_E_I = connect(
            E,
            I,
            E.ids,
            I.ids,
            connect=("bernoulli", {"p": 0.1}),
            on_pre=PreEq_AMPA,
            delay=delay_AMPA,
        )
        S_I_E = connect(
            I,
            E,
            I.ids,
            E.ids,
            connect=("bernoulli", {"p": 0.1}),
            on_pre=PreEq_GABA,
            delay=delay_GABA,
        )
        S_I_I = connect(
            I,
            I,
            I.ids,
            I.ids,
            connect=("bernoulli", {"p": 0.1}),
            on_pre=PreEq_GABA,
            delay=delay_GABA,
        )

        # initialize vars and monitor

        E.set_pop_var(
            variable="v_s",
            value=draw_uniform(a=eL_p / mV - 5.0, b=eL_p / mV + 5.0, size=E.size) * mV,
        )
        E.set_pop_var(
            variable="v_d",
            value=draw_uniform(a=eL_p / mV - 5.0, b=eL_p / mV + 5.0, size=E.size) * mV,
        )

        E.monitor(E.ids, ["v_s"])
        E.monitor_spike(E.ids)
        E.monitor_rate()

        I.set_pop_var(
            variable="v",
            value=draw_uniform(a=eL_i / mV - 5.0, b=eL_i / mV + 5.0, size=I.size) * mV,
        )

        I.monitor(I.ids, ["v"])
        I.monitor_spike(I.ids)
        I.monitor_rate()

        # external inputs

        # poisson device groups
        PE = PoissonDeviceGroup(size=E.size, rate=rate_poisson_e* kHz)
        #PE.monitor_spike(PE.ids)

        PI = PoissonDeviceGroup(size=I.size, rate=rate_poisson_i * kHz)
        #PI.monitor_spike(PI.ids)

        # synapses
        S_PE_E = connect(
            PE,
            E,
            PE.ids,
            E.ids,
            connect=("one2one", {}),
            on_pre=PreEq_AMPA_pois,
            delay=delay_AMPA,
        )
        S_PI_I = connect(
            PI,
            I,
            PI.ids,
            I.ids,
            connect=("one2one", {}),
            on_pre=PreEq_AMPA_pois,
            delay=delay_AMPA,
        )

        exp.run(sim_time * ms)

In [ ]:

from analysis import ExperimentAnalysis
from plot import ExperimentPlotter

def plot(data_path):
    with FileMap(data_path, mode="read") as f:

        analyzer = ExperimentAnalysis(experiment_data=f, t_start=100)
        analyzer.analyze_instantaneous_rate()
        analyzer.analyze_smoothed_rate(window_size=1.0)
        analyzer.analyze_power_spectral_density()
        analyzer.analyze_power_spectral_density(separate_intervals=True)
        analysis = analyzer.report

        print("Analysis done.")

        plotter = ExperimentPlotter(data=f, analysis=analysis, figsize=(20,10), t_start=100, pop_name_e="E", pop_name_i="I")

        plotter.plot_spike_train()
        plotter.plot_voltages()
        plotter.plot_instantaneous_rate(pop_name=["E", "I"])
        plotter.plot_smoothed_rate(pop_name=["E"])
        plotter.plot_smoothed_rate(pop_name=["I"])

        plotter.draw()

        plotter.show()


        plotter = ExperimentPlotter(data=f, analysis=analysis, figsize=(20,10), sharex=True, t_start=100, pop_name_e="E", pop_name_i="I")
        plotter.plot_power_spectrum(pop_name="E")
        plotter.plot_power_spectrum(pop_name="I", is_i=True)

        plotter.draw()
        plotter.show()



        plotter = ExperimentPlotter(data=f, analysis=analysis, figsize=(20,10), sharex=True, t_start=100, pop_name_e="E", pop_name_i="I")
        plotter.plot_power_spectogram_over_time(pop_name="E")
        plotter.plot_power_spectogram_over_time(pop_name="I")
        plotter.draw()
        plotter.show()

## oscillatory dynamics for varying homogeneous poisson input to E and I population

### parameter combinations in kHz

In [ ]:
rates_e = np.linspace(5.0,7.0,5)
rates_i = np.linspace(6.0,9.0,5)
parameters = list(zip(rates_e, rates_i[::-1]))
print(parameters)

Data of parameter combination (6.5, 6.75) produce plots close to Figs 3.1, 3.2 

In [ ]:
sim_time = 500  
for rpe, rpi in parameters:
    path = os.path.join(base_path, f"exp_osc_rpe_{str(rpe).replace('.', '_')}_rpi_{str(rpi).replace('.', '_')}.h5")
    if os.path.isfile(path):
        print("Skipping rpe: {rpe}, rpi: {rpi} - file already exists - delete it if you want to overwrite")
        continue
    print(f"Running rpe: {rpe}, rpi: {rpi}"+"\n\n"+"="*80 +"\n\n")
    run_exp(path=path, sim_time=sim_time, rate_poisson_e=rpe, rate_poisson_i=rpi)
    print("\n\n" +"="*80)


### plots

In [ ]:
for rpe, rpi in parameters:
    print(os.path.join(base_path, f"exp_osc_rpe_{str(rpe).replace('.', '_')}_rpi_{str(rpi).replace('.', '_')}.h5"))

### simulations

In [ ]:
plot("../data/exp_osc_rpe_5_0_rpi_9_0.h5")

In [ ]:
plot("../data/exp_osc_rpe_5_5_rpi_8_25.h5")

In [ ]:
plot("../data/exp_osc_rpe_6_0_rpi_7_5.h5")

In [ ]:
# simiar to Fig 3.2
plot("../data/exp_osc_rpe_6_5_rpi_6_75.h5")

In [ ]:
plot("../data/exp_osc_rpe_7_0_rpi_6_0.h5")